In [1]:
import sympy as sym
import math as m
import numpy as np
import pandas as pd

import scipy.optimize
from sympy import pprint
from scipy.optimize import fsolve
from scipy.optimize import least_squares
from scipy.optimize import minimize

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import SIC_POVM_functions as sic

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


[(6+0j), (6+0j), (2.4+0j), (2.5999999999999996+1.637849848627056e-17j), (2.287500258169+0j), (2.4+2.2311252215006176e-17j), (2.4-2.576281494458285e-17j), (2.3000000000000003-4.125283626344832e-18j), (2.587500258169+0j)]


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:239: ComplexWarning: Casting complex values to real discards the imaginary part
  retval = _minpack._hybrd(func, x0, args, 1, xtol, maxfev,
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


In [2]:
"""#Class definitions #1
class seed_sol:
    def __init__(self, seed):
        self.seed = seed
    @property
    def initial_guess(self):
        np.random.seed(self.seed)
        initial_guess = np.random.rand(35)  # random initial guess
        return initial_guess
    @property
    def solution(self):
        solution = fsolve(Lagrange_eqs, self.initial_guess)
        return solution
    @property
    def res_list(self):
        res_list = Lagrange_eqs(self.solution)
        return res_list
    @property
    def res(self):
        res = np.sum(np.abs(self.res_list))
        return res
    def full_POVM(self):
        #break solution into 4 parts, remove the last 7 elements
        sol = self.solution
        solution_add = [sol[:7], sol[7:14], sol[14:21], sol[21:28]]     # divide into 4 parts of 7
        POVM_normalized_9d = [list(i) for i in POVM_normalized_5d]         # deep copy 
        for i in range(2):                              # first two vecs are known
            POVM_normalized_9d[i].extend([0] *4)
        for part in solution_add:                       # adding (already normalized) elements from the solution to make the remaining vectors 9d
            for i in range(7):
                # POVM_normalized_9d[i+2].append(part[i])
                POVM_normalized_9d[i+2].append((1/np.sqrt(6))*(part[i]))        # normalizing coming from the fixing of the lagrangian eqns
        return POVM_normalized_9d


class POVM_relations:
    def __init__(self, POVM_vec_list):
        self.POVM_vec_list = np.array(POVM_vec_list)        # making array for easy dot product
        self.inner_products = {}
    def dot_product(self, i, j):
        if i < 0 or j < 0 or i > 8 or j > 8:
            raise ValueError('Bad index, must be integer 0 to 8')
        return np.vdot(self.POVM_vec_list[i], self.POVM_vec_list[j])
    def all(self, clean = True, threshold = 1e-12, absolutes = True):
        if len(self.POVM_vec_list) != 9:
            raise ValueError('The POVM list must have 9 elements')
        for i in range(9):
            for j in range(i, 9):
                key = f'{i+1}{j+1}'
                value = self.dot_product(i, j)
                self.inner_products[key] = value
        #cleaning the dictionary
        rel = self.inner_products
        for key in rel:
            if absolutes == True:
                rel[key] = abs(rel[key])
            if abs(rel[key]) < threshold:
                rel[key] = 0
        return rel
"""

"#Class definitions #1\nclass seed_sol:\n    def __init__(self, seed):\n        self.seed = seed\n    @property\n    def initial_guess(self):\n        np.random.seed(self.seed)\n        initial_guess = np.random.rand(35)  # random initial guess\n        return initial_guess\n    @property\n    def solution(self):\n        solution = fsolve(Lagrange_eqs, self.initial_guess)\n        return solution\n    @property\n    def res_list(self):\n        res_list = Lagrange_eqs(self.solution)\n        return res_list\n    @property\n    def res(self):\n        res = np.sum(np.abs(self.res_list))\n        return res\n    def full_POVM(self):\n        #break solution into 4 parts, remove the last 7 elements\n        sol = self.solution\n        solution_add = [sol[:7], sol[7:14], sol[14:21], sol[21:28]]     # divide into 4 parts of 7\n        POVM_normalized_9d = [list(i) for i in POVM_normalized_5d]         # deep copy \n        for i in range(2):                              # first two vecs ar

In [3]:
#Class definitions #2
class povm_create:                  # creates POVM object given a list of 28 coefficients (the unknown variables)
    def __init__(self, coeffs):     # coeffs is the given list of 28 coefficients, make sure it is unnormalized!
        self.coeffs = coeffs
    @property
    def full_POVM(self):
        #break solution into 4 parts, remove the last 7 elements
        POVM_normalized_5d_np = np.array(POVM_normalized_5d).T      # transpose to get 5x9 array 
        # print(POVM_normalized_5d_np.shape)        
        # print(POVM_normalized_5d_np)        

        sol_56 = np.array(self.coeffs)
        sol = sol_56[:28] + 1j * sol_56[28:]        # combine to make 28 complex      
        sol = sol.reshape(4, 7)                     # reshape to 4x7
        zero_columns = np.zeros((4, 2))              # 2 columns of zeros
        sol_padded_normalized = (1/np.sqrt(6))*np.concatenate((zero_columns, sol), axis=1)
        
        POVM_normalized_9d = np.concatenate((POVM_normalized_5d_np , sol_padded_normalized), axis=0)
        # print(POVM_normalized_9d.shape)
        # print(POVM_normalized_9d)
        return POVM_normalized_9d



class POVM_gram_matrix:
    def __init__(self, POVM_vec_list):
        self.POVM_vec_list = np.array(POVM_vec_list)        # making array for easy dot product
    @property
    def gram_matrix(self):
        povm_matrix = self.POVM_vec_list
        povm_matrix_dagger = np.conjugate(povm_matrix).T

        gram_matrix = povm_matrix_dagger @ povm_matrix
        
        return gram_matrix
    


In [4]:
# creating five lists POVMs

w = m.e**((2/3)*m.pi*(1j))     # third root of unity
POVM_unnormalized = [[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]             # unnormalized POVM direction vectors
# POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # normalized POVM direction vectors
POVM_vec = (1/np.sqrt(2))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # normalized POVM direction vectors

# c4j_list = [2,.5,.5,.5, (-.25-.433013j), -.25, .5, -.25, (-.25-.433013j)]           # fourth elements, not normalized yet. See sic.py for calcs
c4j_list = [2,.5,.5,.5, (-.25-.25j*np.sqrt(3)), (-.25+.25j*np.sqrt(3)), .5, (-.25+.25j*np.sqrt(3)), (-.25-.25j*np.sqrt(3))]           # fourth elements, not normalized yet. See sic.py for calcs
# c5j_list = [0, np.sqrt(15)/2 , 0.38729833462074165, (-0.38729833462074176-0.44721359549995776j), -0.19364916731037082, 0.5809475019311124, -0.38729833462074165, (-0.19364916731037093-0.4472135954999579j), 0.5809475019311124]
c5j_list = [0, np.sqrt(15)/2 , (1/np.sqrt(15))*(3/2), (1/np.sqrt(15))*(2*w**2-.5) ,-(1/np.sqrt(15))*(2+(5/2)*w**2), (1/np.sqrt(15))*(2-w/2), (1/np.sqrt(15))*(2*w-.5), -(1/np.sqrt(15))*(2+(5/2)*w), (1/np.sqrt(15))*(2-.5*w**2)]

POVM_unnormalized_5d = []
for i in range(len(POVM_unnormalized)):
	# Directly append c4j_list[i] and c5j_list[i] to the copies of lists in POVM_unnormalized
	vec_5d_i = POVM_unnormalized[i] + [c4j_list[i], c5j_list[i]]
	POVM_unnormalized_5d.append(vec_5d_i)

POVM_unnormalized_5d[0]    ## unnormalized 5-lists POVM vectors

POVM_normalized_5d = ((1/(6**.5))*(np.array(POVM_unnormalized_5d))).tolist()       # normalized 5-lists POVM vectors

#print and compare
# print(POVM_unnormalized_5d[0])	## UNnormalized 5-lists POVM vectors
# print(POVM_normalized_5d[0])	## normalized 5-lists POVM vectors

# delta_five  = [0.400000000000000, 0.433333333333333, 0.381250043028167, 0.400000000000000, 0.400000000000000, 0.383333333333333, 0.431250043028167]
# delta_five_new = [(0.40000000000000013+0j), (0.4333333333333334-1.5431352694421904e-18j), (0.3812500430281667+0j), (0.40000000000000013-3.0923901048776238e-18j), (0.4+1.898287466587076e-18j), (0.38333333333333347-7.881938927040631e-19j), (0.4312500430281667+3.469446951953614e-18j)]
# print('delta_five_new = ', [np.vdot(POVM_normalized_5d[i], POVM_normalized_5d[i]) for i in range(9)])
delta_five_fixed = [ np.vdot(POVM_normalized_5d[i], POVM_normalized_5d[i]) for i in range(2,9)]
print('delta_five_fixed = ', delta_five_fixed)


delta_five_fixed =  [(0.40000000000000013+0j), (0.4333333333333334-1.5431352694421904e-18j), (0.43333333333333335+1.0728477129103336e-18j), (0.43333333333333346-4.471148278649265e-19j), (0.4333333333333333+2.6275172237666522e-18j), (0.4333333333333335+2.267799535210488e-19j), (0.43333333333333335+6.251554030278836e-19j)]


In [5]:
def Relation_Res(vars_full):     # vars should be a list of 56 elements - 28 real and corresponding to them 28 imaginary
    vars_np = np.array(vars_full)    # convert to array
    
    # complex_vars = [ c + 1j*y for c, y in zip(vars_full[:28], vars_full[28:])]    #create 28 complex variables combining the real and imaginary parts
    # complex_vars = vars_np[:28] + 1j*vars_np[28:]    #create 28 complex variables combining the real and imaginary parts

    # we don;t even need to combine the real and imaginary parts, it is done in the class function itself. so just pass the 56 elements directly
    povm = povm_create(vars_full).full_POVM             # create POVM object and use that to get the POVM list
    povm_gram_matrix = POVM_gram_matrix(povm).gram_matrix         # get the gram matrix of the POVM
    residue_ortho = (1/2)* np.sum(np.abs(povm_gram_matrix - np.eye(9)))    # calculate the residue
    return residue_ortho

In [54]:
# Trying for the new complex variabled version, with 56 variables (NOT using Lagrangian)

# takes ~ 16 sec for 100 iterations
seeds = []

for i in range(300, 5300):
    np.random.seed(i)
    ig = np.random.rand(56)
    sol_317 = minimize(Relation_Res, ig, method='CG', tol=1e-15)
    if i%250 == 0:
        print(i)
    if sol_317.fun < 1e-7:
        seeds.append((i,sol_317.fun))
        print( i, sol_317.fun)
        # print( sol.x, '\n\n')


301 8.106160346151403e-08
302 7.308701427270959e-08
303 6.209077467295147e-08
304 6.078349313770396e-08
305 8.284137357480267e-08
306 7.294910862793166e-08
307 7.443828539688428e-08
308 6.597905585667282e-08
309 6.620397690390861e-08
310 6.9900830101836e-08
311 6.235140216480497e-08
312 7.68193314993818e-08
313 6.461726626374507e-08
314 6.244445751407915e-08
315 8.129389396781071e-08
316 6.138230290013114e-08
317 6.796535843840606e-08
318 6.778523233529312e-08
319 7.947764788540479e-08
320 7.894141313559618e-08
321 5.768424924866765e-08
322 7.071047544811794e-08
323 9.127493937536281e-08
324 8.550751689802569e-08
325 7.924866526443633e-08
326 5.7165600427715364e-08
327 6.551710042742059e-08
328 7.026399567442605e-08
329 5.949801044576599e-08
330 6.599774407746878e-08
331 8.735491931901826e-08
332 6.504291028765125e-08
333 6.925291148702246e-08
334 7.299632571227706e-08
335 5.688435859379166e-08
336 7.315518459148872e-08
337 8.711288582391639e-08
338 6.334527072799585e-08
339 6.40664825

KeyboardInterrupt: 

In [54]:
# s = np.array([-0.06765182,  0.75142385 , 0.07345708, -0.00152039 , 0.01613574 , 0.08847722,  0.2236816  ,-0.99985835 , 
            # 0.99844276,  0.3956108,   0.2906196,   0.99609524, -0.2216533  , 0.89769822 , 1.29019934, -0.02932978,  
            # 0.6434624,  -0.25155432, -0.36271562 , 0.49278895 , 0.11850448,  0.77769661,  0.4958864,   0.75934409,
            #   -0.52682382 , 0.53400026 , 0.16225632,  0.65440968,  0.13707977, -0.87482577, -0.2741254  , 0.05410329 ,
            # 0.92248669,  1.51988402,  1.05259268, -0.06495915,  0.60924219 , 0.87560399 , 1.45188224, -0.49589504, 
            #   0.62984922,  0.11388193,  0.48650703 , 0.19327132 , 0.94407672,  0.83293457,  0.09960532, -0.19215606, 
            # -0.5690733  , 0.25803215 , 0.64642148, -0.71717566,  0.41238326,  0.94021283,  0.57525654 ,-0.81058268])

np.random.seed(900)
ig = np.random.rand(56)
sol_317 = minimize(Relation_Res, ig, method='CG', tol=1e-15)
sol_317 = sol_317.x.flatten()
povm_900 = povm_create(sol_317).full_POVM

# print(np.round(povm_900,4))

gram_900 = POVM_gram_matrix(povm_900).gram_matrix

print(np.round(gram_900,9))


[[ 1.00000000e+00+0.e+00j -0.00000000e+00+0.e+00j -0.00000000e+00+0.e+00j
  -0.00000000e+00+0.e+00j  0.00000000e+00-0.e+00j -0.00000000e+00-0.e+00j
  -0.00000000e+00+0.e+00j -0.00000000e+00-0.e+00j  0.00000000e+00-0.e+00j]
 [-0.00000000e+00+0.e+00j  1.00000000e+00+0.e+00j  0.00000000e+00+0.e+00j
  -0.00000000e+00-0.e+00j  0.00000000e+00+0.e+00j -0.00000000e+00-0.e+00j
   0.00000000e+00-0.e+00j -0.00000000e+00-0.e+00j -0.00000000e+00-0.e+00j]
 [-0.00000000e+00+0.e+00j  0.00000000e+00+0.e+00j  9.99999998e-01-0.e+00j
  -2.00000000e-09+0.e+00j -3.00000000e-09-0.e+00j -2.00000000e-09+1.e-09j
  -2.00000000e-09-1.e-09j -2.00000000e-09-1.e-09j -3.00000000e-09-1.e-09j]
 [-0.00000000e+00-0.e+00j -0.00000000e+00+0.e+00j -2.00000000e-09-0.e+00j
   9.99999996e-01+0.e+00j -2.00000000e-09-1.e-09j -2.00000000e-09-1.e-09j
  -3.00000000e-09-0.e+00j -0.00000000e+00+0.e+00j -2.00000000e-09+0.e+00j]
 [ 0.00000000e+00+0.e+00j  0.00000000e+00-0.e+00j -3.00000000e-09+0.e+00j
  -2.00000000e-09+1.e-09j  9.99999

In [55]:
# (np.arccos(1e-7/30))*180/m.pi   # ~avergae angle between vectors in degrees

89.99999980901407

In [55]:
seeds 

[(301, 8.106160346151403e-08),
 (302, 7.308701427270959e-08),
 (303, 6.209077467295147e-08),
 (304, 6.078349313770396e-08),
 (305, 8.284137357480267e-08),
 (306, 7.294910862793166e-08),
 (307, 7.443828539688428e-08),
 (308, 6.597905585667282e-08),
 (309, 6.620397690390861e-08),
 (310, 6.9900830101836e-08),
 (311, 6.235140216480497e-08),
 (312, 7.68193314993818e-08),
 (313, 6.461726626374507e-08),
 (314, 6.244445751407915e-08),
 (315, 8.129389396781071e-08),
 (316, 6.138230290013114e-08),
 (317, 6.796535843840606e-08),
 (318, 6.778523233529312e-08),
 (319, 7.947764788540479e-08),
 (320, 7.894141313559618e-08),
 (321, 5.768424924866765e-08),
 (322, 7.071047544811794e-08),
 (323, 9.127493937536281e-08),
 (324, 8.550751689802569e-08),
 (325, 7.924866526443633e-08),
 (326, 5.7165600427715364e-08),
 (327, 6.551710042742059e-08),
 (328, 7.026399567442605e-08),
 (329, 5.949801044576599e-08),
 (330, 6.599774407746878e-08),
 (331, 8.735491931901826e-08),
 (332, 6.504291028765125e-08),
 (333, 6.9

In [8]:
# create povm and gram matrix for the seed 301

np.random.seed(301)
ig = np.random.rand(56)
sol_317 = minimize(Relation_Res, ig, method='CG', tol=1e-15)
sol_317 = sol_317.x.flatten()
povm_301 = povm_create(sol_317).full_POVM
gram_301 = POVM_gram_matrix(povm_301).gram_matrix

In [69]:
pprint(sym.Matrix(np.round(gram_301,7)))

⎡1.0   0    0    0    0    0    0    0    0 ⎤
⎢                                           ⎥
⎢ 0   1.0   0    0    0    0    0    0    0 ⎥
⎢                                           ⎥
⎢ 0    0   1.0   0    0    0    0    0    0 ⎥
⎢                                           ⎥
⎢ 0    0    0   1.0   0    0    0    0    0 ⎥
⎢                                           ⎥
⎢ 0    0    0    0   1.0   0    0    0    0 ⎥
⎢                                           ⎥
⎢ 0    0    0    0    0   1.0   0    0    0 ⎥
⎢                                           ⎥
⎢ 0    0    0    0    0    0   1.0   0    0 ⎥
⎢                                           ⎥
⎢ 0    0    0    0    0    0    0   1.0   0 ⎥
⎢                                           ⎥
⎣ 0    0    0    0    0    0    0    0   1.0⎦


### Rough

In [71]:
# Finding if any elements are proportional to w

In [77]:
import numpy as np

def check_proportional_to_w_and_w2(complex_list, tolerance=0.02+0.02j, w=np.exp(2j*np.pi/3)):
    proportional_indices = []
    w2 = w**2
    for idx, num in enumerate(complex_list):
        normalized_num = num / np.abs(num) if np.abs(num) != 0 else 0
        if (abs(normalized_num.real - w.real) <= tolerance.real and abs(normalized_num.imag - w.imag) <= tolerance.imag) or \
           (abs(normalized_num.real + w.real) <= tolerance.real and abs(normalized_num.imag + w.imag) <= tolerance.imag) or \
           (abs(normalized_num.real - w2.real) <= tolerance.real and abs(normalized_num.imag - w2.imag) <= tolerance.imag) or \
           (abs(normalized_num.real + w2.real) <= tolerance.real and abs(normalized_num.imag + w2.imag) <= tolerance.imag):
            proportional_indices.append(idx)
    return proportional_indices

# Example usage with a dummy list of complex numbers
dummy_complex_list = [-w**2, 1+1j, 2+2j, 3+3j, 1+1.01j, -w, 2.01+2j, 3+3.01j, w/2, 1+1.1j, 2.1+2j, 3+3.1j, np.sqrt(6)*(w**2/5), -w**2]

proportional_indices_v4 = check_proportional_to_w_and_w2(dummy_complex_list)
print(proportional_indices_v4)

[0, 5, 8, 12, 13]


In [85]:
# run for the complex solution of seeds list
complex_sol_list  = []
for seed in seeds:
    np.random.seed(seed[0])
    ig = np.random.rand(56)
    sol_317 = minimize(Relation_Res, ig, method='CG', tol=1e-15)
    sol_317 = sol_317.x.flatten()
    sol_317_complex = [sol_317[i] + 1j*sol_317[i+28] for i in range(28)]
    complex_sol_list.append(np.array(sol_317_complex))


In [96]:
# run through the funciton
proportional_indices_list = []
for i in range(len(complex_sol_list)):
    proportional_indices = check_proportional_to_w_and_w2(complex_sol_list[i])
    if len(proportional_indices) > 2:
        proportional_indices_list.append((i, proportional_indices))

#check for one
# proportional_indices = check_proportional_to_w_and_w2(complex_sol_list[4])
# proportional_indices

proportional_indices_list


[(50, [0, 2, 24]), (61, [4, 18, 21]), (64, [0, 3, 8, 23])]

In [99]:
check_proportional_to_w_and_w2(complex_sol_list[64], tolerance=0.005+0.005j)

[3]

### Idea

Basically, after finding the unitary in terms of the measurment 9d vectors, we can check the abstract matrix for our system. From this, we can see how the elements of the matrix are correlated because it is only a 6-parameter object. For example, if the matrix element 1,2 is negative of the element 7,5, well, in that case, we know that 7,5 is fixed according to whatever the already-known matrix element 1,2 is. Eg, U_12 = 1 implies U_75 = -1. So, thus knowing a few of those elements from the 27 unknowns, we can start with initial guesses at those points and see if we can find a solution. This solution should converge while keeping those points the same. Alternatively, we can force this condition by modifying the objective function. We would ensure that when the solver changes these elements, the function is not affected. In other words, the function still incorporates these already known values. The solver is actually changing just a dummy variable.

it is possible though that the desired matrix is not implemented using our particular apparatus of 3 BS and 3 phase shifters because there are too few variables in our physical system (only 6) while there are too many variables in the nuemann measurement vectors.

Oh, but actually we can also combine this problem into a more manageable minimum problem which is to basically make a function or make it combine the function out of the 6 parameters and the 27 parameters/36 parameters of the unknown system such that we are minimizing a combined system of all of these parameters and the objective function is the norm of the difference of the two matrices - the abstract system unitary and the POVM unitary related to the POVM solution. The former matrix is a function of first 6 parameters and the second is a function of the 27/54 variables.



So basically, one big objective function that directily gives us implemenataion.